In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

os.chdir("/content/drive/MyDrive/myvit/covnext_t") # 再跳转到我们刚刚下载好的apex项目文件夹中

In [ ]:
#GTSRB
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/myvit/datasets.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/") # 我使用的解压位置

In [ ]:
#BTsd
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/myvit/BelgiumTSC.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/BelgiumTSC") # 我使用的解压位置

In [ ]:
#TSRD
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/myvit/TSRD_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/TSRD") # 我使用的解压位置

In [ ]:
!nvcc -V
import torch
torch.version.cuda

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


'12.1'

In [ ]:
!pip install timm
!pip install einops
!pip install datasets


In [ ]:
import timm

In [ ]:

model_list = timm.list_models("mobilenetv3*")
print(model_list)


['mobilenetv3_large_075', 'mobilenetv3_large_100', 'mobilenetv3_rw', 'mobilenetv3_small_050', 'mobilenetv3_small_075', 'mobilenetv3_small_100']


In [ ]:
#mobilenetv3_small_050
model = timm.create_model('mobilenetv3_large_100', pretrained = False, num_classes = 53)

In [ ]:
model = timm.create_model('convnext_tiny', pretrained = False, num_classes = 43)

In [ ]:
model = timm.create_model('mobilevit_s', pretrained = False, num_classes = 43)

In [ ]:
#v0
# 计算模型的总参数量
total_params = sum(p.numel() for p in model.parameters())

# 计算可训练的参数量
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

Total Parameters: 4269925
Trainable Parameters: 4269925


In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.metrics import classification_report
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy
import cv2
import numpy as np
import timm
from timm.data.mixup import Mixup
from datasets import load_dataset
from torch.utils.data import DataLoader
from torchvision.transforms import (Compose, Normalize, RandomHorizontalFlip,
                                    RandomResizedCrop, RandomRotation, Resize, ToTensor)
from tqdm.notebook import tqdm
import warnings
import json
warnings.filterwarnings("ignore")
import time
import os
import torch.nn.functional as F
import torchvision.models as models
from einops import rearrange
from torch import nn
from torch.utils.data import random_split

In [ ]:
# 设置全局参数
model_lr = 1e-4
BATCH_SIZE = 32
EPOCHS = 80
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
use_amp=False #是否使用混合精度
CLIP_GRAD=5.0
# 数据预处理7
#p.value.
#t.TEST使用如t检验或Wilcoxon符号秩检验

In [ ]:
classes=54 #TSRD
#classes=43 #GTSRB
#classes=53 #BTSD

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),  # 调整明暗度、对比度、饱和度
    transforms.RandomRotation(degrees=15),  # 在[-15, 15]度范围内随机旋转
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 在x和y方向上各随机平移图像的10%
    transforms.ToTensor()
])

transform_test = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

mixup_fn = Mixup(
    mixup_alpha=0.8, cutmix_alpha=0.0, cutmix_minmax=None,
    prob=0.1, switch_prob=0.5, mode='batch',
    label_smoothing=0.1, num_classes=classes)


In [ ]:
#德国
dataset_train = datasets.ImageFolder('/content/datasets/train', transform=transform)
dataset_test = datasets.ImageFolder('/content/datasets/test', transform=transform_test)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)
#德国
print(dataset_train.class_to_idx)
with open('/content/datasets/class.txt','w') as file:
    file.write(str(dataset_train.class_to_idx))
with open('/content/datasets/class.json','w',encoding='utf-8') as file:
    file.write(json.dumps(dataset_train.class_to_idx))

{'CarefulBikes': 0, 'CarefulChildren': 1, 'CarefulDangerous': 2, 'CarefulPedestrians': 3, 'CarefulSlipping': 4, 'CarefulSnow': 5, 'CarefulTrafficLight': 6, 'ConstructionRoad': 7, 'LeftTurn': 8, 'MainRoutePriority': 9, 'MustGoStraight': 10, 'MustGoStraightOrLeft': 11, 'MustGoStraightOrRight': 12, 'MustLeftSide': 13, 'MustRightSide': 14, 'MustTurnLeft': 15, 'MustTurnRight': 16, 'NeedsToYield': 17, 'NoAccess': 18, 'NoEntry': 19, 'NoOvertaking': 20, 'NoOvertakingRemove': 21, 'NoOvertakingTrucks': 22, 'NoOvertakingTrucksRemove': 23, 'NoTrucks': 24, 'PriorityRoad': 25, 'RightTurn': 26, 'RoadNarrowing': 27, 'SharpTurn': 28, 'SpeedLimit100': 29, 'SpeedLimit120': 30, 'SpeedLimit20': 31, 'SpeedLimit30': 32, 'SpeedLimit50': 33, 'SpeedLimit60': 34, 'SpeedLimit70': 35, 'SpeedLimit80': 36, 'SpeedLimit80Lifted': 37, 'Stop': 38, 'Turntable': 39, 'UnevenRoad': 40, 'Unrestrict': 41, 'WildAnimals': 42}


In [ ]:
#比利时
dataset_train = datasets.ImageFolder('/content/BelgiumTSC/BelgiumTSC/Training', transform=transform)
dataset_test = datasets.ImageFolder('/content/BelgiumTSC/BelgiumTSC/Testing', transform=transform_test)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

print(dataset_train.class_to_idx)
with open('/content/BelgiumTSC/BelgiumTSC/class.txt','w') as file:
    file.write(str(dataset_train.class_to_idx))
with open('/content/BelgiumTSC/BelgiumTSC/class.json','w',encoding='utf-8') as file:
    file.write(json.dumps(dataset_train.class_to_idx))

{'00000': 0, '00001': 1, '00002': 2, '00003': 3, '00004': 4, '00005': 5, '00006': 6, '00007': 7, '00008': 8, '00010': 9, '00012': 10, '00013': 11, '00014': 12, '00016': 13, '00017': 14, '00018': 15, '00019': 16, '00020': 17, '00021': 18, '00022': 19, '00023': 20, '00024': 21, '00025': 22, '00027': 23, '00028': 24, '00029': 25, '00030': 26, '00031': 27, '00032': 28, '00034': 29, '00035': 30, '00037': 31, '00038': 32, '00039': 33, '00040': 34, '00041': 35, '00042': 36, '00043': 37, '00044': 38, '00045': 39, '00046': 40, '00047': 41, '00049': 42, '00051': 43, '00053': 44, '00054': 45, '00055': 46, '00056': 47, '00057': 48, '00058': 49, '00059': 50, '00060': 51, '00061': 52}


In [ ]:
#中国
dataset_train = datasets.ImageFolder('/content/TSRD/TSRD_dataset/Train', transform=transform)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataset_test = datasets.ImageFolder('/content/TSRD/TSRD_dataset/Test', transform=transform_test)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

print(dataset_train.class_to_idx)
with open('/content/TSRD/TSRD_dataset/class.txt','w') as file:
    file.write(str(dataset_train.class_to_idx))
with open('/content/TSRD/TSRD_dataset/class.json','w',encoding='utf-8') as file:
    file.write(json.dumps(dataset_train.class_to_idx))

{'000': 0, '001': 1, '002': 2, '003': 3, '004': 4, '005': 5, '006': 6, '007': 7, '008': 8, '010': 9, '011': 10, '012': 11, '013': 12, '014': 13, '015': 14, '016': 15, '017': 16, '020': 17, '021': 18, '022': 19, '023': 20, '024': 21, '025': 22, '026': 23, '027': 24, '028': 25, '029': 26, '030': 27, '031': 28, '032': 29, '034': 30, '035': 31, '036': 32, '037': 33, '038': 34, '039': 35, '040': 36, '041': 37, '042': 38, '043': 39, '044': 40, '045': 41, '046': 42, '047': 43, '048': 44, '049': 45, '050': 46, '051': 47, '052': 48, '053': 49, '054': 50, '055': 51, '056': 52, '057': 53}


In [ ]:
# 检查训练集和测试集的类别数量
train_classes = dataset_train.class_to_idx
test_classes = dataset_test.class_to_idx

# 打印类别数量
print("Number of classes in training set:", len(train_classes))
print("Number of classes in testing set:", len(test_classes))

# 检查类别数量是否相等
if len(train_classes) == len(test_classes):
    print("Both datasets have the same number of classes.")
else:
    print("Mismatch in the number of classes between datasets.")

# 检查具体的类别是否一致
train_class_set = set(train_classes.keys())
test_class_set = set(test_classes.keys())

if train_class_set == test_class_set:
    print("All classes match between training and testing sets.")
else:
    print("There are differences in class names between the datasets.")
    # 找出不匹配的类别
    only_in_train = train_class_set - test_class_set
    only_in_test = test_class_set - train_class_set
    print("Classes only in training set:", only_in_train)
    print("Classes only in testing set:", only_in_test)

Number of classes in training set: 43
Number of classes in testing set: 43
Both datasets have the same number of classes.
All classes match between training and testing sets.


In [ ]:
#E-MobileViT
def _make_divisible(v, divisor, min_value=None):
	if min_value is None:
		min_value = divisor
	new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
	# Make sure that round down does not go down by more than 10%.
	if new_v < 0.9 * v:
		new_v += divisor
	return new_v


def Conv_BN_ReLU(inp, oup, kernel, stride=1):
	return nn.Sequential(
		nn.Conv2d(inp, oup, kernel_size=kernel, stride=stride, padding=1, bias=False),
		nn.BatchNorm2d(oup),
		nn.ReLU6(inplace=True)
	)


def conv_1x1_bn(inp, oup):
	return nn.Sequential(
		nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
		nn.BatchNorm2d(oup),
		nn.ReLU6(inplace=True)
	)


class PreNorm(nn.Module):
	def __init__(self, dim, fn):
		super().__init__()
		self.norm = nn.LayerNorm(dim)
		self.fn = fn

	def forward(self, x, **kwargs):
		return self.fn(self.norm(x), **kwargs)


class FeedForward(nn.Module):
	def __init__(self, dim, hidden_dim, dropout=0.):
		super().__init__()
		self.ffn = nn.Sequential(
			nn.Linear(dim, hidden_dim),
			nn.SiLU(),
			nn.Dropout(dropout),
			nn.Linear(hidden_dim, dim),
			nn.Dropout(dropout)
		)

	def forward(self, x):
		return self.ffn(x)


class Attention(nn.Module):
	def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
		super().__init__()
		inner_dim = dim_head * heads
		project_out = not (heads == 1 and dim_head == dim)

		self.heads = heads
		self.scale = dim_head ** -0.5

		self.attend = nn.Softmax(dim=-1)
		self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

		self.to_out = nn.Sequential(
			nn.Linear(inner_dim, dim),
			nn.Dropout(dropout)
		) if project_out else nn.Identity()

	def forward(self, x):
		qkv = self.to_qkv(x).chunk(3, dim=-1)
		q, k, v = map(lambda t: rearrange(t, 'b p n (h d) -> b p h n d', h=self.heads), qkv)

		dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
		attn = self.attend(dots)
		out = torch.matmul(attn, v)
		out = rearrange(out, 'b p h n d -> b p n (h d)')
		return self.to_out(out)


class Transformer(nn.Module):
	def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
		super().__init__()
		self.layers = nn.ModuleList([])
		for _ in range(depth):
			self.layers.append(nn.ModuleList([
				PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout)),
				PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
			]))

	def forward(self, x):
		for attn, ff in self.layers:
			x = attn(x) + x
			x = ff(x) + x
		return x

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.ca(x)
        return out * self.sa(out)

class MV2Block(nn.Module):
    def __init__(self, inp, oup, stride=1, expand_ratio=4):
        super(MV2Block, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                # nn.ReLU6(inplace=True),
                nn.SiLU(),
                # CBAM 注意力机制放置在 3x3 卷积层之后
                CBAM(hidden_dim),  # 添加 CBAM 注意力机制
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                # nn.ReLU6(inplace=True),
                nn.SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                # nn.ReLU6(inplace=True),
                nn.SiLU(),
                # CBAM 注意力机制放置在 3x3 卷积层之后
                CBAM(hidden_dim),  # 添加 CBAM 注意力机制
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)

# 实现ELA注意力机制
class ELAAttention(nn.Module):
    def __init__(self, channel, kernel_size):
        super().__init__()
        self.kernel_size = kernel_size
        self.pad = kernel_size // 2
        self.conv = nn.Conv1d(channel, channel, kernel_size, padding=self.pad, groups=channel, bias=False)
        self.gn = nn.GroupNorm(16, channel)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, h, w = x.size()
        x_h = torch.mean(x, dim=3, keepdim=True).view(b, c, h)
        x_w = torch.mean(x, dim=2, keepdim=True).view(b, c, w)

        x_h = self.sigmoid(self.gn(self.conv(x_h))).view(b, c, h, 1)
        x_w = self.sigmoid(self.gn(self.conv(x_w))).view(b, c, 1, w)
        return x * x_h * x_w



class MobileViTBlock(nn.Module):
    def __init__(self, dim, depth, channel, kernel_size, patch_size, mlp_dim, dropout=0.):
        super().__init__()
        self.ph, self.pw = patch_size

        self.conv1 = Conv_BN_ReLU(channel, channel, kernel_size)
        self.conv2 = conv_1x1_bn(channel, dim)

        # Transformer module includes attention mechanism internally
        self.transformer = Transformer(dim, depth, 1, 32, mlp_dim, dropout)

        # ELA Attention module
        self.ela_attention = ELAAttention(channel, kernel_size)

        # We will apply Transformer and ELAAttention in parallel to the feature map
        # Then, we fuse their outputs along with the original feature map from conv2

        # Since we're fusing three feature maps: original, Transformer output, and ELA output,
        # we'll have 3 * channel output channels before reducing it back to the original channel size.
        self.conv3 = conv_1x1_bn(dim * 2 + channel, channel)  # Adjust for the tripled channel size
        self.conv4 = Conv_BN_ReLU(channel, channel, kernel_size)  # Process fused feature map

    def forward(self, x):
        y = x.clone()

        # Local representations
        x_local = self.conv1(x)
        x_local = self.conv2(x_local)

        # 计算原始特征图的空间维度
        _, _, h, w = x_local.shape

        # Transformer representations
        # 调整 x_local 的形状以适应 Transformer 的输入要求
        x_transformed = rearrange(x_local, 'b d (h ph) (w pw) -> b (h w) (ph pw) d', ph=self.ph, pw=self.pw)
        x_transformed = self.transformer(x_transformed)
        # 将 Transformer 的输出重新排列回原始的空间维度
        x_transformed = rearrange(x_transformed, 'b (h w) (ph pw) d -> b d (h ph) (w pw)', ph=self.ph, pw=self.pw, h=h // self.ph, w=w // self.pw)

        # ELA Attention representations
        x_ela = self.ela_attention(y)

        # Fusion of features from both attentions and the original
        # 确保x_ela, x_transformed, 和 x_local 在空间维度上一致
        x_fused = torch.cat((x_local, x_ela, x_transformed), 1)

        # Reduce channel size and process
        x_reduced = self.conv3(x_fused)
        x_final = self.conv4(x_reduced)

        return x_final




class MobileVit_v2(nn.Module):
	def __init__(self, image_size, dims, channels, num_classes, expansion=4, kernel_size=3, patch_size=(2, 2)):
		super().__init__()
		ih, iw = image_size
		ph, pw = patch_size
		assert ih % ph == 0 and iw % pw == 0

		L = [2, 4, 3]

		self.conv1 = Conv_BN_ReLU(3, channels[0], kernel=3, stride=2)

		self.mv2 = nn.ModuleList([])
		self.mv2.append(MV2Block(channels[0], channels[1], 1, expansion))
		self.mv2.append(MV2Block(channels[1], channels[2], 2, expansion))
		self.mv2.append(MV2Block(channels[2], channels[3], 1, expansion))
		self.mv2.append(MV2Block(channels[2], channels[3], 1, expansion))  # Repeat
		self.mv2.append(MV2Block(channels[3], channels[4], 2, expansion))
		self.mv2.append(MV2Block(channels[5], channels[6], 2, expansion))
		self.mv2.append(MV2Block(channels[7], channels[8], 2, expansion))

		self.mvit = nn.ModuleList([])
		self.mvit.append(MobileViTBlock(dims[0], L[0], channels[5], kernel_size, patch_size, int(dims[0] * 2)))
		self.mvit.append(MobileViTBlock(dims[1], L[1], channels[7], kernel_size, patch_size, int(dims[1] * 4)))
		self.mvit.append(MobileViTBlock(dims[2], L[2], channels[9], kernel_size, patch_size, int(dims[2] * 4)))

		self.conv2 = conv_1x1_bn(channels[-2], channels[-1])

		self.pool = nn.AvgPool2d(ih // 32, 1)
		self.fc = nn.Linear(channels[-1], num_classes, bias=False)

	def forward(self, x):
		x = self.conv1(x)
		x = self.mv2[0](x)

		x = self.mv2[1](x)
		x = self.mv2[2](x)
		x = self.mv2[3](x)  # Repeat

		x = self.mv2[4](x)
		x = self.mvit[0](x)

		x = self.mv2[5](x)
		x = self.mvit[1](x)

		x = self.mv2[6](x)
		x = self.mvit[2](x)
		x = self.conv2(x)

		x = self.pool(x).view(-1, x.shape[1])
		x = self.fc(x)
		return x


def mobilevit_xxs_v2():
	dims = [64, 80, 96]
	channels = [16, 16, 24, 24, 48, 48, 64, 64, 80, 80, 320]
	return MobileVit_v2((256, 256), dims, channels, num_classes=54, expansion=2)


def mobilevit_xs_v2():
	dims = [96, 120, 144]
	channels = [16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384]
	return MobileVit_v2((256, 256), dims, channels, num_classes=54)


def mobilevit_s_v2():
	dims = [144, 192, 240]
	channels = [16, 32, 64, 64, 96, 96, 128, 128, 160, 160, 640]
	return MobileVit_v2((256, 256), dims, channels, num_classes=54)

In [ ]:
#MobileViT
def _make_divisible(v, divisor, min_value=None):
	if min_value is None:
		min_value = divisor
	new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
	# Make sure that round down does not go down by more than 10%.
	if new_v < 0.9 * v:
		new_v += divisor
	return new_v


def Conv_BN_ReLU(inp, oup, kernel, stride=1):
	return nn.Sequential(
		nn.Conv2d(inp, oup, kernel_size=kernel, stride=stride, padding=1, bias=False),
		nn.BatchNorm2d(oup),
		nn.ReLU6(inplace=True)
	)


def conv_1x1_bn(inp, oup):
	return nn.Sequential(
		nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
		nn.BatchNorm2d(oup),
		nn.ReLU6(inplace=True)
	)


class PreNorm(nn.Module):
	def __init__(self, dim, fn):
		super().__init__()
		self.norm = nn.LayerNorm(dim)
		self.fn = fn

	def forward(self, x, **kwargs):
		return self.fn(self.norm(x), **kwargs)


class FeedForward(nn.Module):
	def __init__(self, dim, hidden_dim, dropout=0.):
		super().__init__()
		self.ffn = nn.Sequential(
			nn.Linear(dim, hidden_dim),
			nn.SiLU(),
			nn.Dropout(dropout),
			nn.Linear(hidden_dim, dim),
			nn.Dropout(dropout)
		)

	def forward(self, x):
		return self.ffn(x)


class Attention(nn.Module):
	def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
		super().__init__()
		inner_dim = dim_head * heads
		project_out = not (heads == 1 and dim_head == dim)

		self.heads = heads
		self.scale = dim_head ** -0.5

		self.attend = nn.Softmax(dim=-1)
		self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

		self.to_out = nn.Sequential(
			nn.Linear(inner_dim, dim),
			nn.Dropout(dropout)
		) if project_out else nn.Identity()

	def forward(self, x):
		qkv = self.to_qkv(x).chunk(3, dim=-1)
		q, k, v = map(lambda t: rearrange(t, 'b p n (h d) -> b p h n d', h=self.heads), qkv)

		dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
		attn = self.attend(dots)
		out = torch.matmul(attn, v)
		out = rearrange(out, 'b p h n d -> b p n (h d)')
		return self.to_out(out)


class Transformer(nn.Module):
	def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
		super().__init__()
		self.layers = nn.ModuleList([])
		for _ in range(depth):
			self.layers.append(nn.ModuleList([
				PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout)),
				PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
			]))

	def forward(self, x):
		for attn, ff in self.layers:
			x = attn(x) + x
			x = ff(x) + x
		return x


class MV2Block(nn.Module):
	def __init__(self, inp, oup, stride=1, expand_ratio=4):
		super(MV2Block, self).__init__()
		assert stride in [1, 2]

		hidden_dim = round(inp * expand_ratio)
		self.identity = stride == 1 and inp == oup

		if expand_ratio == 1:
			self.conv = nn.Sequential(
				# dw
				nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
				nn.BatchNorm2d(hidden_dim),
				# nn.ReLU6(inplace=True),
				nn.SiLU(),
				# pw-linear
				nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
				nn.BatchNorm2d(oup),
			)
		else:
			self.conv = nn.Sequential(
				# pw
				nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
				nn.BatchNorm2d(hidden_dim),
				# nn.ReLU6(inplace=True),
				nn.SiLU(),
				# dw
				nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
				nn.BatchNorm2d(hidden_dim),
				# nn.ReLU6(inplace=True),
				nn.SiLU(),
				# pw-linear
				nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
				nn.BatchNorm2d(oup),
			)

	def forward(self, x):
		if self.identity:
			return x + self.conv(x)
		else:
			return self.conv(x)


class MobileViTBlock(nn.Module):
	def __init__(self, dim, depth, channel, kernel_size, patch_size, mlp_dim, dropout=0.):
		super().__init__()
		self.ph, self.pw = patch_size

		self.conv1 = Conv_BN_ReLU(channel, channel, kernel_size)
		self.conv2 = conv_1x1_bn(channel, dim)

		self.transformer = Transformer(dim, depth, 1, 32, mlp_dim, dropout)

		self.conv3 = conv_1x1_bn(dim, channel)
		self.conv4 = Conv_BN_ReLU(2 * channel, channel, kernel_size)

	def forward(self, x):
		y = x.clone()

		# Local representations
		x = self.conv1(x)
		x = self.conv2(x)

		# Global representations
		_, _, h, w = x.shape
		x = rearrange(x, 'b d (h ph) (w pw) -> b (ph pw) (h w) d', ph=self.ph, pw=self.pw)
		x = self.transformer(x)
		x = rearrange(x, 'b (ph pw) (h w) d -> b d (h ph) (w pw)', h=h // self.ph, w=w // self.pw, ph=self.ph,
		              pw=self.pw)

		# Fusion
		x = self.conv3(x)
		x = torch.cat((x, y), 1)
		x = self.conv4(x)
		return x


class MobileVit(nn.Module):
	def __init__(self, image_size, dims, channels, num_classes, expansion=4, kernel_size=3, patch_size=(2, 2)):
		super().__init__()
		ih, iw = image_size
		ph, pw = patch_size
		assert ih % ph == 0 and iw % pw == 0

		L = [2, 4, 3]

		self.conv1 = Conv_BN_ReLU(3, channels[0], kernel=3, stride=2)

		self.mv2 = nn.ModuleList([])
		self.mv2.append(MV2Block(channels[0], channels[1], 1, expansion))
		self.mv2.append(MV2Block(channels[1], channels[2], 2, expansion))
		self.mv2.append(MV2Block(channels[2], channels[3], 1, expansion))
		self.mv2.append(MV2Block(channels[2], channels[3], 1, expansion))  # Repeat
		self.mv2.append(MV2Block(channels[3], channels[4], 2, expansion))
		self.mv2.append(MV2Block(channels[5], channels[6], 2, expansion))
		self.mv2.append(MV2Block(channels[7], channels[8], 2, expansion))

		self.mvit = nn.ModuleList([])
		self.mvit.append(MobileViTBlock(dims[0], L[0], channels[5], kernel_size, patch_size, int(dims[0] * 2)))
		self.mvit.append(MobileViTBlock(dims[1], L[1], channels[7], kernel_size, patch_size, int(dims[1] * 4)))
		self.mvit.append(MobileViTBlock(dims[2], L[2], channels[9], kernel_size, patch_size, int(dims[2] * 4)))

		self.conv2 = conv_1x1_bn(channels[-2], channels[-1])

		self.pool = nn.AvgPool2d(ih // 32, 1)
		self.fc = nn.Linear(channels[-1], num_classes, bias=False)

	def forward(self, x):
		x = self.conv1(x)
		x = self.mv2[0](x)

		x = self.mv2[1](x)
		x = self.mv2[2](x)
		x = self.mv2[3](x)  # Repeat

		x = self.mv2[4](x)
		x = self.mvit[0](x)

		x = self.mv2[5](x)
		x = self.mvit[1](x)

		x = self.mv2[6](x)
		x = self.mvit[2](x)
		x = self.conv2(x)

		x = self.pool(x).view(-1, x.shape[1])
		x = self.fc(x)
		return x


def mobilevit_xxs():
	dims = [64, 80, 96]
	channels = [16, 16, 24, 24, 48, 48, 64, 64, 80, 80, 320]
	return MobileVit((256, 256), dims, channels, num_classes=1000, expansion=2)


def mobilevit_xs():
	dims = [96, 120, 144]
	channels = [16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384]
	return MobileVit((256, 256), dims, channels, num_classes=1000)


def mobilevit_s():
	dims = [144, 192, 240]
	channels = [16, 32, 64, 64, 96, 96, 128, 128, 160, 160, 640]
	return MobileVit((256, 256), dims, channels, num_classes=54)

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from scipy.stats import ttest_rel
import torchvision.datasets as datasets

# 检查GPU可用性并设置设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 加载模型并转移到GPU
model1 = torch.load('/content/drive/MyDrive/mv2/mv2_2_TSRD_m55_0.972.pth').to(device)

# 确保模型在评估模式
model1.eval()


MobileVit_v2(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (mv2): ModuleList(
    (0): MV2Block(
      (conv): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): SiLU()
        (6): CBAM(
          (ca): ChannelAttention(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (max_pool): AdaptiveMaxPool2d(output_size=1)
            (fc): Sequential(
              (0): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
         

In [ ]:
# 检查GPU可用性并设置设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 加载模型并转移到GPU
model2 = torch.load('/content/drive/MyDrive/myvit/mv0_t/modelv0_TSRD_m49_0.899.pth').to(device)


# 确保模型在评估模式
model2.eval()

MobileVit(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (mv2): ModuleList(
    (0): MV2Block(
      (conv): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): SiLU()
        (6): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): MV2Block(
      (conv): Sequential(
        (0): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1)

In [ ]:
#tsrd

dataset_test = datasets.ImageFolder('/content/TSRD/TSRD_dataset/Test', transform=transform_test)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 用于收集每个模型在每个折上的性能
results_model1 = []
results_model2 = []

def evaluate_model(model, loader):
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# K 折交叉验证
for train_index, test_index in kfold.split(dataset_test):
    print(f'Fold {5}: test_index {test_index[:5]}')  # 打印一部分索引以进行检查
    test_subsampler = Subset(dataset_test, test_index)

    test_loader = torch.utils.data.DataLoader(test_subsampler, batch_size=32, shuffle=False)

    # 训练模型 (这里假设模型已经训练完成，直接评估)
    acc_model1 = evaluate_model(model1, test_loader)
    acc_model2 = evaluate_model(model2, test_loader)

    results_model1.append(acc_model1)
    results_model2.append(acc_model2)

# T检验
t_stat, p_value = ttest_rel(results_model1, results_model2)
print(f'T-statistic: {t_stat}, P-value: {p_value}')


Fold 5: test_index [23 29 30 32 44]
Fold 5: test_index [ 2  6 15 18 31]
Fold 5: test_index [ 0  3  5  7 10]
Fold 5: test_index [ 4  9 11 16 17]
Fold 5: test_index [ 1  8 13 14 20]
T-statistic: 12.349177318962608, P-value: 0.00024708618161364696


In [ ]:
##GTSRB
dataset_test = datasets.ImageFolder('/content/datasets/test', transform=transform_test)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 用于收集每个模型在每个折上的性能
results_model1 = []
results_model2 = []

def evaluate_model(model, loader):
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# K 折交叉验证
for train_index, test_index in kfold.split(dataset_test):
    print(f'Fold {5}: test_index {test_index[:5]}')  # 打印一部分索引以进行检查
    test_subsampler = Subset(dataset_test, test_index)

    test_loader = torch.utils.data.DataLoader(test_subsampler, batch_size=32, shuffle=False)

    # 训练模型 (这里假设模型已经训练完成，直接评估)
    acc_model1 = evaluate_model(model1, test_loader)
    acc_model2 = evaluate_model(model2, test_loader)

    results_model1.append(acc_model1)
    results_model2.append(acc_model2)

# T检验
t_stat, p_value = ttest_rel(results_model1, results_model2)
print(f'T-statistic: {t_stat}, P-value: {p_value}')

Fold 5: test_index [ 0  3  8 14 17]
Fold 5: test_index [10 12 20 23 29]
Fold 5: test_index [15 26 27 28 34]
Fold 5: test_index [ 2  6  7 18 22]
Fold 5: test_index [ 1  4  5  9 11]
T-statistic: 3.172049109212676, P-value: 0.03379140128372741


In [ ]:
##btsd
dataset_test = datasets.ImageFolder('/content/BelgiumTSC/BelgiumTSC/Testing', transform=transform_test)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 用于收集每个模型在每个折上的性能
results_model1 = []
results_model2 = []

def evaluate_model(model, loader):
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# K 折交叉验证
for train_index, test_index in kfold.split(dataset_test):
    print(f'Fold {5}: test_index {test_index[:5]}')  # 打印一部分索引以进行检查
    test_subsampler = Subset(dataset_test, test_index)

    test_loader = torch.utils.data.DataLoader(test_subsampler, batch_size=32, shuffle=False)

    # 训练模型 (这里假设模型已经训练完成，直接评估)
    acc_model1 = evaluate_model(model1, test_loader)
    acc_model2 = evaluate_model(model2, test_loader)

    results_model1.append(acc_model1)
    results_model2.append(acc_model2)

# T检验
t_stat, p_value = ttest_rel(results_model1, results_model2)
print(f'T-statistic: {t_stat}, P-value: {p_value}')

Fold 5: test_index [25 29 43 44 48]
Fold 5: test_index [ 8 13 18 20 23]
Fold 5: test_index [ 2  6 10 14 15]
Fold 5: test_index [0 3 5 7 9]
Fold 5: test_index [ 1  4 11 16 19]
T-statistic: 2.060488785479736, P-value: 0.10839243802227273
